In [1]:
import mcdm
import numpy as np
from pymcdm.methods import TOPSIS
from pymcdm.methods import VIKOR
from pymcdm.methods import PROMETHEE_II
from pymcdm.helpers import rankdata,rrankdata
from pymcdm import weights as mcdm_weights

In [2]:
from prettytable import from_csv
with open("liver_perform.tsv") as fp:
    mytable = from_csv(fp)
print(mytable)

+----------------------+----------+-----------+--------+-------------+---------+-------------+-------------+-------------+--------------+--------------+
|                      | ACCURACY | PRECISION | RECALL |     AUC     | F1SCORE |   FP_RATE   |   FN_RATE   | SPECIFICITY | HAMMING LOSS | ENTROPY LOSS |
+----------------------+----------+-----------+--------+-------------+---------+-------------+-------------+-------------+--------------+--------------+
| LOGISTIC REGRESSION  |   0.7    |    0.65   |  0.7   | 0.671238245 |   0.67  | 0.827586207 | 0.136363636 | 0.172413793 | 0.299145299  | 10.33227662  |
|    DECISION TREE     |   0.61   |    0.62   |  0.61  | 0.484326019 |   0.61  |  0.75862069 | 0.272727273 |  0.24137931 | 0.393162393  | 13.57949834  |
| GAUSSIAN NAIVE BAYES |   0.5    |    0.83   |  0.5   | 0.684169279 |   0.5   |      0      | 0.670454545 |      1      | 0.504273504  | 17.41698981  |
|         SVM          |   0.62   |    0.59   |  0.62  | 0.587774295 |   0.6   | 0

In [3]:
x_matrix, alt_names = mcdm.load("liver_perform.tsv", delimiter="\t", skiprows=1, labeled_rows=True)
alts = np.array(x_matrix, dtype='float')
weight = [0.10909090908971901,0.16363636363821488, 0.18181818182165294,0.12727272727352062,0.14545454545314052,0.0363636363625124,0.0727272727239339,0.0545454545446777,0.09090909091046283,0.018181818182165292]
weights=np.array(weight)
types = np.array([1,1,1,1,1,-1,-1,1,-1,-1])

In [4]:
topsis = TOPSIS()
# Determine preferences and ranking for alternatives
pref1 = topsis(alts, weights, types)
ranking1= rrankdata(pref1)

for r, p in zip(ranking1, pref1):
    print(r, p)


2.0 0.6761495648342509
6.0 0.461929008487331
4.0 0.5315008728143463
5.0 0.49084418738883473
3.0 0.570943377927817
1.0 0.7736498701119502
7.0 0.4473022726976816


In [5]:
vikor= VIKOR()
pref2 = vikor(alts, weights, types)
ranking2= rankdata(pref2)

for r, p in zip(ranking2, pref2):
    print(r, p)

2.0 0.3065270048761727
5.0 0.7193538755653403
3.0 0.6476163191401723
6.0 0.7552356956691617
4.0 0.6516358555370442
1.0 0.0
7.0 1.0


In [6]:
promethe=PROMETHEE_II('usual')
p = np.random.rand(x_matrix.shape[1]) / 2
q = np.random.rand(x_matrix.shape[1]) / 2 + 0.5
pref3 = promethe(alts, weights, types,p=p,q=q)
ranking3 = rrankdata(pref3)

for r, p in zip(ranking3, pref3):
    print(r, p)

2.0 0.42424242424282643
7.0 -0.3878787878799504
4.0 -0.13636363636187604
5.0 -0.24242424242438576
3.0 0.1333333333335152
1.0 0.4909090909097961
6.0 -0.28181818181992563


In [7]:
from prettytable import PrettyTable
print("#########################LIVER DATASET#######################")
columns = ["Model", "TOPSIS","Value","VIKOR","VALUES","PROMETHEE","VALUES"]
Table = PrettyTable()
Table.add_column(columns[0], ["LOGISTIC REGRESSION ", "DECISION TREE", "GAUSSIAN NAIVE BAYES",
                       "SVM ", "NEURAL NETWORK", "RANDOM FOREST ", "LINEAR MODEL"])
Table.add_column(columns[1], ranking1)
Table.add_column(columns[2], pref1)
Table.add_column(columns[3], ranking2)
Table.add_column(columns[4], pref2)
Table.add_column(columns[5], ranking3)
Table.add_column(columns[6], pref3)

print(Table)

#########################LIVER DATASET#######################
+----------------------+--------+---------------------+-------+--------------------+-----------+----------------------+
|        Model         | TOPSIS |        Value        | VIKOR |       VALUES       | PROMETHEE |        VALUES        |
+----------------------+--------+---------------------+-------+--------------------+-----------+----------------------+
| LOGISTIC REGRESSION  |  2.0   |  0.6761495648342509 |  2.0  | 0.3065270048761727 |    2.0    | 0.42424242424282643  |
|    DECISION TREE     |  6.0   |  0.461929008487331  |  5.0  | 0.7193538755653403 |    7.0    | -0.3878787878799504  |
| GAUSSIAN NAIVE BAYES |  4.0   |  0.5315008728143463 |  3.0  | 0.6476163191401723 |    4.0    | -0.13636363636187604 |
|         SVM          |  5.0   | 0.49084418738883473 |  6.0  | 0.7552356956691617 |    5.0    | -0.24242424242438576 |
|    NEURAL NETWORK    |  3.0   |  0.570943377927817  |  4.0  | 0.6516358555370442 |    3.0    |  

In [8]:
ans=mcdm.rank(x_matrix, alt_names=alt_names,is_benefit_x=[1,1,1,1,1,-1,-1,1,-1,-1],n_method="Linear2", w_vector=weight, s_method="SAW") 
model=[]
value=[]
for x in ans:
    model.append(x[0])
    value.append(x[1])

print("#########################SAW#######################")
columns = ["Model","Value"]
Table = PrettyTable()
Table.add_column(columns[0], model)
Table.add_column(columns[1], value)

print(Table) 

#########################SAW#######################
+----------------------+---------------------+
|        Model         |        Value        |
+----------------------+---------------------+
|    RANDOM FOREST     |  0.6960411813184537 |
| LOGISTIC REGRESSION  |  0.6053870014766419 |
| GAUSSIAN NAIVE BAYES |  0.5780667416887352 |
|     LINEAR MODEL     |  0.5496323732125483 |
|    NEURAL NETWORK    | 0.47256153108159304 |
|         SVM          |  0.4417250691160956 |
|    DECISION TREE     |  0.4076006802141997 |
+----------------------+---------------------+


In [9]:
ans=mcdm.rank(x_matrix,alt_names=alt_names,is_benefit_x=[1,1,1,1,1,-1,-1,1,-1,-1], w_vector=weight,n_method="Linear1",s_method="MEW")
model=[]
value=[]
for x in ans:
    model.append(x[0])
    value.append(x[1])

print("#########################MEW#######################")
columns = ["Model","Value"]
Table = PrettyTable()
Table.add_column(columns[0], model)
Table.add_column(columns[1], value)

print(Table)

#########################MEW#######################
+----------------------+--------------------+
|        Model         |       Value        |
+----------------------+--------------------+
|    RANDOM FOREST     | 0.7509533184392336 |
| LOGISTIC REGRESSION  | 0.6701319352673497 |
|    DECISION TREE     | 0.6650994576858991 |
|         SVM          | 0.6501814187033655 |
| GAUSSIAN NAIVE BAYES |        0.0         |
|    NEURAL NETWORK    |        0.0         |
|     LINEAR MODEL     |        0.0         |
+----------------------+--------------------+
